# 0 背景

机器学习的应用主要得益于**有效的统计模型**和**大规模可扩展的学习系统**。从GBDT这种集成树模型（Ensemble Tree Model）提出以来就在机器学习应用领域取得了很好的效果。

本文提出了名为XGBoost的一种可扩展的用于树增强学习的机器学习系统（A scalable machine learning system for tree boosting）。

XGBoost的创新点主要包括了算法和系统两个方面：
- 算法：
    - 一种新的处理稀疏数据的树学习算法
    - Weighted Quantile Sketch（一种处理带权数据集寻找候选切分点的方法）
    - 学习目标中增加正则项，使得学习目标更加完备
- 系统：
    - 并行化分布式计算加速了模型训练
    - 使用核外计算（Out-of-core Computation）使得可以用单机处理大规模数据

# 1 算法

## 1.1 正则化学习目标（Regularized Learning Objective）

给定数据集$\mathcal{D}=\{(x_i,y_i)\}$，树集成模型使用K个加性函数去预测结果$\hat{y_i} = \phi(x_i) = \sum_{k=1}^K f_k(x_i)$，其中$f_k \in \mathcal{F}$，$\mathcal{F}=\{f(x)=w_{q(x)}\}$

即每个基学习器$f_k(x)$是一个回归树（CART），它通过$q(x)$将样本x映射到树中的唯一一个叶子结点，再赋予其对应的叶子权重$w$。由于回归树每个叶子结点的权重$w_i$是连续值，因此最终的预测结果可以看做是样本在每个基学习器中叶子结点的权重$w_i$之和，即$\hat{y_i}=\sum_{k=1}^K w_{q_k(x_i)}$

在XGBoost中，为了学习一系列的基学习器集合$\mathcal{F}=\{f_k\}$，其学习目标为最小化**带有正则项的目标函数**：

$$\mathcal{L}(\phi)=\sum_i l(\hat{y_i}, y_i)+\sum_k \Omega(f_k)$$

其中，$$\Omega(f)=\gamma T +\frac{1}{2}\lambda||\omega||^2$$

具体而言，目标函数包含两部分：
- 每个样本的误差。由损失函数$l(\cdot)$衡量预测值$\hat{y}$和真实值$y$之间的差别。注意，这里的$l(\cdot)$必须是二阶可微凸函数
- 模型复杂度惩罚项。这一项由每个基学习器$f_k$的复杂度来衡量，包含了当前基学习器$f_k$的叶子数$T$和每个叶子的权重$\omega$的模

> 作者在文中提到正则项可以使得最终模型学习的权重更加平滑且避免过拟合。尽管在Regularized greedy forest(RGF)中也有正则项，但XGBoost中的正则项更简单且更容易并行化。

## 1.2 学习方法

上面的优化目标函数中包含了基学习器作为参数，不能够在欧式空间中使用传统的优化方式求解，因此使用了一种**加性方法（additive manner）**求解。

假设我们当前训练第t轮的基学习器$f_t$，则我们需要找到一个最优的学习器$f_t$去最小化损失函数：

$$\mathcal{L}^{(t)}=\sum_{i=1}^n l(y_i, \hat{y_i}^{(t-1)}+f_t(x_i))+\Omega(f_t)$$

上述加性方法是一种贪心算法，即寻找当前能够最小化损失函数的学习器。二阶近似（Second-order Approximation）可以用来优化这个目标函数，即将损失函数$\mathcal{L}^{(t)}(x)$在$x=\hat{y_i}^{(t-1)}$处进行二阶泰勒展开：

$$\mathcal{L}^{(t)}\approx \sum_{i=1}^n [l(y_i, \hat{y_i}^{(t-1)}) + g_i f_t(x_i) + \frac{1}{2}h_i f_t^2(x_i)]+\Omega(f_t)$$

其中，$g_i=\partial_{\hat{y_i}^{(t-1)}}l(y_i, \hat{y_i}^{(t-1)})$，$h_i=\partial^2_{\hat{y_i}^{(t-1)}}l(y_i, \hat{y_i}^{(t-1)})$

## 1.3 函数简化

将二阶泰勒展开后的损失函数中常数项去掉，可以得到：

$$\mathcal{\tilde{L}}^{(t)}=\sum_{i=1}^n [g_i f_t(x_i)+\frac{1}{2}h_i f_t^2(x_i)]+\Omega(f_t)$$

重新用另外一种方式来重写上面函数，定义$I_j=\{i|q(x_i)=j\}$代表属于叶子结点$j$的所有样本集合，那么对于同属于一个叶子结点$j$的样本来说，其当前基学习器的学习值$f_t(x_i)$都是$w_j$，则损失函数可以重写为：

$$\mathcal{\tilde{L}}^{(t)}=\sum_{j=1}^T[(\sum_{i\in I_j}g_i) w_j + (\sum_{i\in I_j} \frac{1}{2} h_i) w_j^2] + \gamma T + \frac{1}{2}\lambda \sum_{j=1}^T w_j^2\\
= \sum_{j=1}^T[(\sum_{i\in I_j}g_i) w_j + \frac{1}{2}(\sum_{i\in I_j} h_i + \lambda) w_j^2] + \gamma T$$

## 1.4 参数求解

根据上面的简化公式，我们待学习的参数包括了$f_t$的**结构**与**参数**两个部分。假设当前模型结构$q(x)$固定，此时每个叶子结点的最优参数$w$可求解为：

$$w_j^*=-\frac{\sum_{i\in I_j} g_i}{\sum_{i\in I_j}h_i+\lambda}$$

将$w_j^*$带回损失函数可得：

$$\mathcal{\tilde{L}}^{(t)}=-\frac{1}{2}\sum_{j=1}^T \frac{(\sum_{i\in I_j}g_i)^2}{\sum_{i\in I_j}h_i+\lambda }+\gamma T$$

上述式子可以用来评估当前给定的树结构$q(x)$的损失和误差。最理想的状态下，我们可以穷尽遍历所有可能的基学习器$f_t$，根据上式计算每个候选学习器的损失，选出最好的。但这种方法在现实中是不可能的。因此，XGBoost采用了一种贪心学习算法来确定当前$f_t$的树结构$q(x)$：

- 将当前所有样本视为一个树结点
- 假设分裂后的左右叶子结点的样本集合分别为$I_L$和$I_R$。计算根据当前特征的当前分裂点结算后的损失降低度：$\mathcal{L}_{split}=\mathcal{L}_I-(\mathcal{L}_{I_L}+\mathcal{L}_{I_R})=\frac{1}{2}\sum_{j=1}^T[\frac{(\sum_{i\in I_L}g_i)^2}{\sum_{i\in I_L}h_i+\lambda} + \frac{(\sum_{i\in I_R}g_i)^2}{\sum_{i\in I_R}h_i+\lambda} - \frac{(\sum_{i\in I}g_i)^2}{\sum_{i\in I}h_i+\lambda}]-\gamma$

论文中具体的子树结构分裂算法：

![](https://raw.githubusercontent.com/NELSONZHAO/paper-learning-and-notes/master/%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0/pictures/xgboost-algo1.png)

第一层遍历所有特征$k=1$ to $d$，第二层按照当前遍历的特征$k$值对所有样本进行排序，不断地计算最大损失的降低程度。最终迭代完所有的特征与分裂点。

时间复杂度为$O(d)\times O(n\log n)$。其中d为特征维度；$O(n\log n)$是因为对于每个特征都要重新排序样本。

> 其实就是当前分裂时，寻找能够最大可能将左右叶子结点样本的梯度之差分裂开的方式；当损失函数是MSE时，一阶梯度就等于样本的残差

## 1.5 其它创新点

1. 权重参数衰减（Shrinkage）：Shrinkage类似于神经网络中的学习率，即每次迭代一个基学习器，对叶子结点的参数$w_j$乘一个因子$\eta$，$\phi(x_i)^{(t)}=\phi(x_i)^{(t-1)}+\eta f_t(x_i)$；Shrinkage可以有效减少每个子模型的影响，给后续的基学习器更大的学习空间。
2. 学习RF一样使用column sub-sampling。不仅可以有效防止过拟合，还可以加速计算并行化

# 2 分裂搜索算法（Split Finding Algorithms）

在XGBoost算法中，主要分为了宏观层面的算法和微观层面的算法。

- 宏观上：采用前向加性算法与学习衰减来组合各个基学习器的结果
- 微观上：通过贪心搜索分裂点确定每一个基学习器结构

在微观层面的算法方面，由于计算资源有限，因此实际中采用了许多手法来去近似求解基学习器的最优结构。

## 2.1 近似搜索算法

在第1部分的基学习器分裂算法中，其穷尽了每个特征所有可能的分裂点，来寻找具有最大增益（最大损失降低度）的分裂特征及其分裂点，这种算法称为**Exact Greedy Algorithm**。

考虑到现实中不可能穷尽所有可能性，因此作者提出了一种方法：
1. 根据特征的分布（百分位点）提供候选分裂点
2. 算法根据百分位点将连续特征映射到离散bin中，通过统计特征寻找最优解

作者总结了上述算法的两个变体：
- Global Variant：在基学习器生成之初，根据特征分布提供所有候选可能的分裂点；
- Local Variant：在每一次分裂时，根据特征分布提供所有候选可能的分裂点；

Global Variant的**优点**在于，在当前基学习器开始建立之初，计算了所有候选的分裂点，因此在当前整棵树的构建过程中，不需要额外的步骤，直接从内存中读取分裂点即可；**缺点**是由于后续的分裂都基于开始给的这些候选值，不会重新提供或者提炼新的可能分裂点，因此Global需要尽可能多的候选分裂点选择；

Local Variant的**优点**在于每一次分裂都会重新基于当前结点的样本提供新的可能分裂点，因此它更可能也更适合去生长更深的树；**缺点**是相比于Global方式，其计算复杂度比较高。

> 在这里根据个人理解说一下原文中提到的Local会Refines candidates的意思。首先，假设我们的数据集中只有一个特征，即用户年龄。在生长当前的基学习器时，Global根据年龄分布计算出分位点并且离散化后，产生了5个候选分裂点：{18, 25, 35, 45, 60}，假设在根节点分裂时，分裂点18的增益最大，即分为了左子树(age<18)和右子树(age>=18)，此时在左子树上，所有样本的年龄都在18岁以下，没有了候选分裂点，左子树停止生长；而对于Local来说，当前分裂结束后，它会重新根据当前左子树的样本(age<18)的年龄分布重新计算分位点，例如{5, 10, 15}，使得左子树可以继续分裂。

## 2.2 Weighted Quantile Sketch

作者提出了一种基于加权数据集寻找最优分裂点的方式。

对于当前第k个feature，我们可以构造数据集$\mathcal{D}_k=\{(x_{1k},h_1), (x_{2k}, h_2), \cdots, (x_{nk}, h_n)\}$，并且定义一个排序函数$r_k$如下：

$$r_k(z)=\frac{1}{\sum_{(x, h)\in \mathcal{D}_k}h}\sum_{(x, h)\in \mathcal{D}_k, x < z} h$$（z是特征本身的取值）

目标是要寻找到最优分裂点集合$\{s_{k1}, s_{k2}, \cdots, s_{kl}\}$，使得分裂点满足条件：

$$|r_k(s_k, j)-r_k(s_k,j+1)|\lt \epsilon, s_{k1}=\min_i x_{ik}, s_{kl}=\max_i x_{ik}$$

相当于给每个样本加了二阶梯度$h_i$的权重后，寻找分裂点。这是因为损失函数的近似项可以化简为：

$$\mathcal{\tilde{L}}\approx \sum_{i=1}^n[g_if_t(x_i)+\frac{1}{2}h_i f_t^2(x_i)]+\Omega(f_t)\\=\sum_{i=1}^n \frac{1}{2}h_i(f_t(x_i)-(-\frac{g_i}{h_i}))^2+\Omega(f_t)+constant$$

因此可以将目标函数看做是预测值$f_t(x_i)$和目标值$-\frac{g_i}{h_i}$的加权平方误差。

对于大规模数据集很难找到满足上述条件的分裂点，当每个样本等权重时，Quantile Sketch可以解决分裂点搜索问题；但当样本带权后，需要用Weighted Quantile Sketch进行解决。

> 如果目标函数中损失函数$l(\cdot)=\frac{1}{2}(\hat{y}-y)^2$时，对于所有样本来说二阶梯度$h_i$都是无差别的，都是一个常数。此时样本相当于等权，上面的公式就可以看做是$f_t(x_i)$和目标值$-g_i$（负梯度，也是残差）的平方误差。

> 当在做二分类问题时，如果使用logloss，即$l(\hat{y}, y) = -y \log \hat{y} - (1-y)\log (1-\hat{y})$，对于样本i来说，有：

> $$g_i=\hat{y}-y$$
> $$h_i=\hat{y}*(1-\hat{y}) \\ 
where\ \hat{y} = \sigma(z)=\frac{1}{1+e^{-z}}$$

> 此时，可以看到二阶梯度$h_i$对样本起到了加权作用，$h_i$的取值分布在一个最大值为1/4的倒U二次函数上，因此当预测的$\hat{y}$越接近于真实值$y$时，$h_i$越小；当预测$\hat{y}$越接近于0.5，即模型无区分度时，$h_i$最大，使得模型在选在分裂点时会考虑样本本身的权重，加大了预测不准确样本的权重。

> 参考链接：https://blog.csdn.net/anshuai_aw1/article/details/82970489#_604

## 2.3 稀疏感知分裂搜索（Sparsity-aware Split Finding）

树模型中往往有很多稀疏数据，包括：
- missing values
- 高频率的0值出现
- One Hot Encoding后的特征

因此作者提出了一种简单的统一处理此类稀疏特征的分裂方法：

![](https://raw.githubusercontent.com/NELSONZHAO/paper-learning-and-notes/master/%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0/pictures/xgboost-algo3.png)

从算法可以看出，基本思路很简单，通过尝试分别将在当前特征上具有缺失的样本放置到左右子树的方法确定更合适的位置：
- 将所有缺失样本放在右子树计算增益
- 将所有缺失样本放在左子树计算增益
- 缺失值样本属于增益大的对应子树

> 特征工程启发：考虑到XGBoost计算特征值缺失样本的默认分裂方向是基于其增益，这个增益的计算方式又是基于样本的梯度$g_i$和$h_i$，进一步地，梯度由样本的目标值决定。因此对于某些空值，如需人工填充，应该将其填充为和这些缺失值样本目标一致的那些样本对应的特征均值。

# 3 系统优化